<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Run GriddedCCP AR

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [5]:
args = [parse_arguments("GridedCCP_AR celeba32 --starting-run 0".split()),
        parse_arguments("GridedCCP_AConfR celeba32 --starting-run 0".split()),
       parse_arguments("GridedCCP_AR mnist --starting-run 0".split()),
        parse_arguments("GridedCCP_AConfR mnist --starting-run 0".split())]

In [6]:
executor=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24, 
                           #partition="dev", 
                           cpus_per_task=10, 
                           mem='32GB',  
                          )

In [7]:
job = executor.map_array(Run(), args)

In [20]:
j = job[0]
job

[SlurmJob<job_id=17767962_0, task_id=0, state="RUNNING">,
 SlurmJob<job_id=17767962_1, task_id=0, state="RUNNING">,
 SlurmJob<job_id=17767962_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=17767962_3, task_id=0, state="COMPLETED">]

In [21]:
for j in job:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2019-09-17 03:01:24,459) - Starting with JobEnvironment(job_id=17767962_0, hostname=learnfair0321, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2019-09-17 03:01:24,459) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/17767962_0/17767962_0_submitted.pkl

--- Training celeba32/GridedCCP_AR/run_0 ---

  epoch    train_loss    valid_loss    cp        dur
-------  ------------  ------------  ----  ---------
      1       -2.4188       -2.3192     +  1025.1561
      2       -2.8518       -1.7310        775.8017
      3       -3.0064       -2.2201        795.8687
      4       -3.0552       -2.2061        789.5654
      5       -3.0794       -2.5011     +  807.9256
      6       -3.1131       -2.3397        780.4694
      7       -3.1242       -1.9951        809.0077
      8       -3.1408       -2.5359     +  765.3742
      9       -3.1348       -2.4476        795.8630
     10       -3.1617       -2.2191

In [16]:
for j in job:
    print("--------------------------------")
    print(j.stderr())

--------------------------------

--------------------------------

--------------------------------

--------------------------------

